# Multi-GPU training

<div class="alert alert-info">

<b>Summary</b>
    
We parallelize a Transformer layer with data, tensor, and sequence parallelism. We also demonstrate how a Transformer layer can be wrapped with PyTorch's Fully Sharded Data Parallel strategy.

</div>

A variety of parallelism strategies can be used to enable multi-GPU training of Transformer models, often based on different approaches to distribute their $\text{sequence_length} \times \text{batch_size} \times \text{hidden_size}$ activation tensors. In this section, we will build on the GPT Transformer layer from the [quickstart guide](quickstart.ipynb) to demonstrate each of these parallelism strategies.

To show this in action, let's first initialize NCCL process group:

In [ ]:
# Configure parallel groups
import os
import torch
LOCAL_RANK = int(os.getenv("LOCAL_RANK", "0"))
WORLD_SIZE = int(os.getenv("WORLD_SIZE", "1"))
world_group = torch.distributed.init_process_group(
    "nccl",
    init_method="file:///tmp/rdzv",
    world_size=WORLD_SIZE,
    rank=LOCAL_RANK,
)
data_parallel_group = torch.distributed.new_group(ranks=[LOCAL_RANK], backend="nccl")
tensor_parallel_group = torch.distributed.new_group(ranks=[LOCAL_RANK], backend="nccl")

Note that Transformer Engine requires that each distributed process corresponds to exactly one GPU. In addition, we initialize the process groups for both data and tensor parallelism with the same GPUs to keep this example simple.

In practice, there are multiple factors that can affect the optimal parallel layout: the system hardware, the network topology, usage of other parallelism schemes like pipeline parallelism. A rough rule-of-thumb is to interpret the GPUs as a 2D grid with dimensions of $\text{num_nodes} \times \text{gpus_per_node}$. The rows are tensor-parallel groups and the columns are data-parallel groups.

The distributed code shown in this section needs to be executed with PyTorch's [torchrun (elastic launch)](https://pytorch.org/docs/stable/elastic/run.html) utility. Below is an example for launching a script on a single node with 4 GPUs:
```bash
$ torchrun --standalone --nnodes=1 --nprocs_per_node=4 script.py $SCRIPT_ARGS
```

The elastic launch utility spawns the requested number of processes on the specified number of nodes, and sets the `LOCAL_RANK` and `WORLD_SIZE` environment variables on every process that are required to initialize the process groups. For further guidance on running with multiple GPUs, please consult the documentation for [torch.distributed](https://pytorch.org/docs/stable/distributed.html).

## Data Parallelism

The most common approach to parallel training distributes along the $\text{batch_size}$ dimension. By storing duplicate copies of the model on each GPU, the forward and backward passes of the training step can be done independently, followed by a gradient synchronization.

Enabling data parallelism with Transformer Engine is similar to enabling data parallelism with standard PyTorch models: simply wrap the modules with [torch.nn.parallel.DistributedDataParallel](https://pytorch.org/docs/stable/generated/torch.nn.parallel.DistributedDataParallel.html). FP8 training requires extra synchronization for the scaling factors, so the data-parallel process group must also be passed to the [fp8_autocast](../api/pytorch.rst#transformer_engine.pytorch.fp8_autocast) context manager.

In [ ]:
# Construct layer
basic_transformer = te.TransformerLayer(
    hidden_size,
    ffn_hidden_size,
    num_attention_heads,
    params_dtype=dtype,
    device='cuda',
)
data_parallel_transformer = torch.nn.parallel.DistributedDataParallel(
    basic_transformer,
    process_group=data_parallel_group,
)

# Training step
with te.fp8_autocast(enabled=True, fp8_recipe=fp8_recipe, fp8_group=data_parallel_group):
    y = data_parallel_transformer(x, attention_mask=None)
y.backward(dy)

# Measure step time
utils.speedometer(
    parallel_transformer,
    x,
    dy,
    forward_kwargs = { "attention_mask": None },
    fp8_autocast_kwargs = {
        "enabled": True,
        "fp8_recipe": fp8_recipe,
        "fp8_group": data_parallel_group,
    },
)

## Tensor and Sequence Parallelism

A more advanced approach is to distribute the activation tensors along the $\text{hidden_size}$ dimension. This allows us to scale past the limits of data parallelism (typically $\text{hidden_size} > \text{batch_size}$) and to reduce the per-GPU memory usage (since model parameters are also distributed), but it also incurs the overhead of communicating activation tensors between GPUs at every step. For a more detailed explanation, please see the [Megatron-LM paper](https://arxiv.org/pdf/1909.08053.pdf).

In addition, sequence parallelism distributes along the $\text{sequence_length}$ dimension. This can be used when tensor parallelism is enabled in order to parallelize operations that run outside the tensor-parallel region (e.g. layer norm). For more details, please see [this paper](https://arxiv.org/pdf/2205.05198.pdf).

Transformer Engine modules have native support for tensor and sequence parallelism. If the user provides a process group for tensor parallelism, the modules will distribute the data and perform communication internally. If sequence parallelism is enabled, it will be applied for operations that are not amenable to tensor parallelism and it will use the tensor-parallel process group. In this case, the tensor parallel group must also be passed to the **fp8_group** argument in the [fp8_autocast](../api/pytorch.rst#transformer_engine.pytorch.fp8_autocast) context manager, either directly or as a subset of a larger distributed group.

In [ ]:
# Construct layer
tensor_parallel_transformer = te.TransformerLayer(
    hidden_size,
    ffn_hidden_size,
    num_attention_heads,
    set_parallel_mode=True,
    tp_group=tensor_parallel_group,
    sequence_parallel=True,
    params_dtype=dtype,
    device='cuda',
)
data_tensor_parallel_transformer = torch.nn.parallel.DistributedDataParallel(
    tensor_parallel_transformer,
    process_group=data_parallel_group,
)

# Training step
with te.fp8_autocast(enabled=True, fp8_recipe=fp8_recipe, fp8_group=data_parallel_group):
    y = data_tensor_parallel_transformer(x, attention_mask=None)
y.backward(dy)

# Measure step time
utils.speedometer(
    data_tensor_parallel_transformer,
    x,
    dy,
    forward_kwargs = { "attention_mask": None },
    fp8_autocast_kwargs = {
        "enabled": True,
        "fp8_recipe": fp8_recipe,
        "fp8_group": data_parallel_group,
    },
)

## Fully Sharded Data Parallel (FSDP)

Transformer Engine is also supports PyTorch's [torch.distributed.fsdp.FullyShardedDataParallel](https://pytorch.org/docs/stable/fsdp.html) utility for parallelism.

When FSDP wraps a PyTorch model, it combines all the module parameters in the model into a `FlatParameter` that is then distributed over all the GPUs in the process group. Each FSDP-wrapped module only sees a view of this `FlatParameter` that corresponds to that module's local parameters on each GPU. The modules themselves execute forward and backward passes exclusively locally. Meanwhile, the FSDP wrapper is responsible for performing the all-gather before the forward and backward passes to ensure every module has the right weights, and the reduce-scatter after the backward pass to make sure the gradient accumulates correctly.

Using FSDP with Transformer Engine requires creating Transformer Engine modules without any inherent parallelism. For a Transformer layer, this means we only initialize a basic transformer without any process groups or sequence parallelism.

In [ ]:
# Construct layer
basic_transformer = te.TransformerLayer(
    hidden_size,
    ffn_hidden_size,
    num_attention_heads,
    params_dtype=dtype,
    device='cuda',
)

wrap_policy = functools.partial(
    torch.distributed.fsdp.wrap.transformer_auto_wrap_policy,
    transformer_layer_cls={te.TransformerLayer},
)
fsdp_transformer =